# Alchemix Code Review
## Intro
This document will contain the main information on the unpicking of the alchemix code. Additional/ supporting documents can be put in the 'alchemixStudy' folder.
This will serve as a central reference of our understanding of the alchemix smart contracts.
## Key questions
While understanding the code, we should pay special attention to the following, as they will be useful in our project:
* How are Yearn yields tracked? - this will be key for us as we must monitor yields coming in from yearn, to be able to show users their renumeration for performing rebalancing actions.
* Security - which security features have been implemented? Which seem different to standard (perhaps whitelisting)? Would these be useful for us?
* Which contracts can we leverage? - contracts such as wEth gateway and yearn token adaptor should be useful to us and we can implement them directly in a diamond pattern. Are there any others?
### Yearn Yields
function _preemptivelyHarvest(line 1002 AlchemistV2.sol)  
![preemptively harvest definition](./IMG/_preemptiveHarvest.png "preemptively harvest")  
Worth checking how yearn yield works - do number of yvault tokens increase over time?

### YieldTokenParams
A struct defined in IAlchemistV2State.sol defining yield token parameters.
It is inherited by IAlchemistV2 which is inherited by AlchemistV2.sol.
It kind of acts as a bit of a glossary of terms:  
![yield token parameters](./IMG/yieldTokenParams.png "yield token parameters") 


## Resources/ References
[Alchemix contracts outline](https://alchemix-finance.gitbook.io/v2/)  
[Alchemix V2 Contracts Github](https://github.com/alchemix-finance/v2-contracts)  
[Alchemix user docs](https://alchemix-finance.gitbook.io/user-docs/)  
[OpenAI prompt design docs](https://beta.openai.com/docs/guides/completion/prompt-design)  
